In [ ]:
import os

import logging

from snowflake.snowpark import Session
 
# Set up logging

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger()
 
# Retrieve Snowflake credentials from environment variables

account = os.getenv('SNOWFLAKE_ACCOUNT')

user = os.getenv('SNOWFLAKE_USER')

password = os.getenv('SNOWFLAKE_PASSWORD')

role = os.getenv('SNOWFLAKE_ROLE')

warehouse = os.getenv('SNOWFLAKE_WAREHOUSE')

database = os.getenv('SNOWFLAKE_DATABASE')

schema = os.getenv('SNOWFLAKE_SCHEMA')
 
# Snowpark session configuration

connection_parameters = {

    "account": account,

    "user": user,

    "password": password,

    "role": role,

    "warehouse": warehouse,

    "database": database,

    "schema": schema

}
 
# This will no longer be used because Snowflake provides the session in the stored procedure.

# session = Session.builder.configs(connection_parameters).create()
 
logger.info("Using provided Snowpark session.")
 
def my_stored_procedure(session: Session):

    try:

        # Set role to ACCOUNTADMIN

        logger.info("Setting role to ACCOUNTADMIN...")

        session.sql("USE ROLE ACCOUNTADMIN").collect()

        logger.info("Role set to ACCOUNTADMIN successfully.")

        # Create the table

        logger.info("Creating table RAW_CO2.Daily_Measurements...")

        create_table_query = """

        CREATE OR REPLACE TABLE RAW_CO2.Daily_Measurements (

            date STRING,

            co2_ppm FLOAT

        );

        """

        session.sql(create_table_query).collect()

        logger.info("Table RAW_CO2.Daily_Measurements created successfully.")

        # Grant privileges on the table to ACCOUNTADMIN role

        logger.info("Granting privileges on table to ACCOUNTADMIN...")

        grant_privileges_query = """

        GRANT ALL PRIVILEGES ON TABLE RAW_CO2.Daily_Measurements TO ROLE ACCOUNTADMIN;

        """

        session.sql(grant_privileges_query).collect()

        logger.info("Privileges granted to ACCOUNTADMIN successfully.")

        # Copy data from the stage into the table

        logger.info("Copying data from the stage into RAW_CO2.Daily_Measurements...")

        copy_query = """

        COPY INTO RAW_CO2.Daily_Measurements

            FROM @RAW_CO2.CO2_EXTERNAL_STAGE

            FILE_FORMAT = (

                TYPE = 'CSV' 

                SKIP_HEADER = 1

                FIELD_OPTIONALLY_ENCLOSED_BY = '"'

            )

            ON_ERROR = 'CONTINUE';

        """

        session.sql(copy_query).collect()

        logger.info("Data copied into RAW_CO2.Daily_Measurements successfully.")

        # Create a stream on the table

        logger.info("Creating stream on RAW_CO2.Daily_Measurements...")

        create_stream_query = """

        CREATE OR REPLACE STREAM RAW_CO2.DAILY_MEASUREMENTS_STREAM 

        ON TABLE RAW_CO2.Daily_Measurements;

        """

        session.sql(create_stream_query).collect()

        logger.info("Stream RAW_CO2.DAILY_MEASUREMENTS_STREAM created successfully.")

        # Optionally, verify if the table, privileges, and stream were created successfully

        logger.info("Table, privileges, and stream are created successfully.")

    except Exception as e:

        logger.error(f"An error occurred: {str(e)}")

        raise
 
# No need to create session manually as Snowflake provides it

# Just call the stored procedure with the provided session from Snowflake
 
# my_stored_procedure(session)  # The session will be automatically passed when Snowflake calls this procedure
 
my_stored_procedure(session)

 